Libraries 

In [1]:
import sys
import json
import time
import requests
import os
import pandas as pd
import numpy as np
from pydub import AudioSegment
import csv
import wave
import numpy as np
import pandas as pd
import glob
import librosa
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import signal
from scipy.io import wavfile
from scipy.fftpack import fft
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils import data
import chardet
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.backend_bases import RendererBase
from transformers import BertModel, BertTokenizer
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
#from .utils import load_state_dict_from_url
from torch.hub import load_state_dict_from_url

__all__ = ['AlexNet', 'alexnet']


model_urls = {
    'alexnet': 'https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth',
}

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',truncation=True)

d:\OSG-AA-Products\Speech-Emotion-Recognition\SER\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Audio Features Extraction 

In [2]:
def audio2spectrogram(filepath):
  window_size=40
  step_size=20
  eps=1e-10
  samplerate, test_sound  = wavfile.read(filepath,mmap=True)
  nperseg = int(round(window_size * samplerate / 1e3))
  noverlap = int(round(step_size * samplerate / 1e3))
  freqs, _, spec = signal.spectrogram(test_sound ,
                                    fs=samplerate,
                                    window='hann',
                                    nperseg=nperseg,
                                    noverlap=noverlap,
                                    detrend=False)
  spectrogram = np.log(spec.T.astype(np.float32) + eps)
  return spectrogram


def get_3d_spec(spectrogram, moments=None):
  N_CHANNELS = 3
  if moments is not None:
        (base_mean, base_std, delta_mean, delta_std,
             delta2_mean, delta2_std) = moments
  else:
        base_mean, delta_mean, delta2_mean = (0, 0, 0)
        base_std, delta_std, delta2_std = (1, 1, 1)
        h, w = spectrogram.shape
        right1 = np.concatenate([spectrogram[:, 0].reshape((h, -1)), spectrogram], axis=1)[:, :-1]
        delta = (spectrogram - right1)[:, 1:]
        delta_pad = delta[:, 0].reshape((h, -1))
        delta = np.concatenate([delta_pad, delta], axis=1)
        right2 = np.concatenate([delta[:, 0].reshape((h, -1)), delta], axis=1)[:, :-1]
        delta2 = (delta - right2)[:, 1:]
        delta2_pad = delta2[:, 0].reshape((h, -1))
        delta2 = np.concatenate([delta2_pad, delta2], axis=1)
        base = (spectrogram - base_mean) / base_std
        delta = (delta - delta_mean) / delta_std
        delta2 = (delta2 - delta2_mean) / delta2_std
        stacked = [arr.reshape((h, w, 1)) for arr in (base, delta, delta2)]
  return np.concatenate(stacked, axis=2)

Alexnet and Modified Alexnet Model Class

In [3]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.num_classes=num_classes
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((12, 12))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        print('features',x.shape)
        return x
def alexnet(pretrained=False, progress=True, **kwargs):
    model = AlexNet(**kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls['alexnet'],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model

class ModifiedAlexNet(nn.Module):
    def __init__(self, num_classes=5):
        super(ModifiedAlexNet, self).__init__()
        self.num_classes=num_classes
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256, num_classes),
        )
        self.Sigmoid = nn.Sigmoid

    def forward(self, x):
        x = self.features(x)
        x=torch.flatten(x, start_dim=2)
        x=torch.sum(x, dim=2)
        x=self.classifier(x)
        return x
   
def modifiedAlexNet(pretrained=False, progress=True, **kwargs):
    model_modified = ModifiedAlexNet(**kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls['alexnet'],
                                              progress=progress)
        model_modified.load_state_dict(state_dict)
    return model_modified



outputs_text= []
def hook_text(module, input, output):
    outputs_text.clear()
    outputs_text.append(output)
    return None

outputs_audio= []
def hook_audio(module, input, output):
    outputs_audio.clear()
    outputs_audio.append(output)
    return None

class CombinedAudioTextModel(nn.Module):
    def __init__(self, num_classes=5):
        super(CombinedAudioTextModel, self).__init__()
        self.num_classes=num_classes
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

        self.text_model=torch.load('models/model_text_best.pt',map_location=torch.device('cpu'))
        self.audio_model=torch.load('models/model_audio_best.pt',map_location=torch.device('cpu'))

        self.text_model.bert.pooler.register_forward_hook(hook_text)
        self.audio_model.features.register_forward_hook(hook_audio)

        for param in self.text_model.parameters():
          param.requires_grad = False
        for param in self.audio_model.parameters():
          param.requires_grad = False

        self.dropout = nn.Dropout(.5)
        self.linear = nn.Linear(1024, num_classes)

        self.Sigmoid = nn.Sigmoid

    def forward(self,text,audio):
        self.text_model(text)
        self.audio_model(audio)
        audio_embed=outputs_audio[0]
        text_embed=outputs_text[0]
        audio_embed=torch.flatten(audio_embed, start_dim=2)#a1,a2,a3......al{a of dim c} 
        audio_embed=torch.sum(audio_embed, dim=2)
        concat_embded=torch.cat((text_embed,audio_embed),1)
        x=self.dropout(concat_embded)
        x=self.linear(x)
        return x


In [4]:
audio_model = ModifiedAlexNet()
audio_model=torch.load('models/model_audio_best.pt',map_location=torch.device('cpu'))

Model Loading

In [5]:
model=torch.load('models/Combined_model_audio_text.pt',map_location=torch.device('cpu'))
model.eval()
model.to('cpu')
torch.save(model,"model.pt")

In [23]:
model=torch.load('model.pt',map_location=torch.device('cpu'))
model.eval()
model.to('cpu')

CombinedAudioTextModel(
  (text_model): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features

In [6]:
def SER(audio_filepath,start_time,end_time,utter):
  indextolabel={0: 'ang', 1: 'hap', 3: 'neu', 2: 'sad', 4: 'exc'}
  fname = os.path.splitext(os.path.basename(audio_filepath))[0]
  os.makedirs("crop_audio", exist_ok=True)
  output_path = 'crop_audio/'
  t1,t2 =start_time,end_time

  newAudio = AudioSegment.from_file(audio_filepath)
  newAudio = newAudio[t1:t2]
  newAudio.export(output_path+str(t1)+'_'+str(t2)+'_'+fname+".wav", format="wav")
  file_path = output_path+str(t1)+'_'+str(t2)+'_'+fname+".wav"

  y, sr = librosa.load(file_path, sr = 16000, mono=True)
  y = y * 32767 / max(0.01, np.max(np.abs(y)))
  audio_filepath = file_path.split(".wav")[0] + ".wav"
  wavfile.write(file_path, sr, y.astype(np.int16))

  #print(file_path)
  try:
    if (audio2spectrogram(file_path)).shape[0]!=0:
      prob = None
      preds = None
      spector=audio2spectrogram(file_path)
      spector=get_3d_spec(spector)
      npimg = np.transpose(spector,(2,0,1))
      input_tensor=torch.tensor(npimg)
      sprectrome = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
      input_ids = torch.tensor(tokenizer.encode(utter, add_special_tokens=True,truncation=True)).unsqueeze(0)
      with torch.no_grad():
          if (sprectrome.shape[2]>10):
              output = model(input_ids,sprectrome)
              m = nn.Softmax(dim=1)
              probs = m(output)
              preds = torch.argmax(probs)
              prob = torch.max(probs)
              #print("Predicted class is {} with {}% probability".format(indextolabel[preds.item()], round(prob.item() * 100,2)))
              #print()
              probs = probs.tolist()[0]
              emotion ={}
          for i in range(len(probs)):
              emotion[indextolabel[i]] = round(probs[i] * 100,2)
              emotions = emotion
          #print(file_path)
          os.remove(file_path)    
          return emotions
    elif (audio2spectrogram(file_path)).shape[0]==0:
      #print(file_path)
      os.remove(file_path)
  except:
    #print(file_path)
    os.remove(file_path)

In [11]:
df = pd.read_csv("videoplayback1.csv")
df.head()

,file_name,speaker,sequence,utter,start_time,end_time
0,videos/videoplayback1.mp4,A,0,"Hi, there.",10300,10610
1,videos/videoplayback1.mp4,B,1,My name is Leah. I'm one of the junior doctors...,10642,16760
2,videos/videoplayback1.mp4,C,2,Yes.,16867,17180
3,videos/videoplayback1.mp4,A,3,So.,17227,17390
4,videos/videoplayback1.mp4,D,4,It's Camilla. Camilla.,17407,18565


In [12]:
df['Emotion']=df.apply(lambda x: SER(x['file_name'],x['start_time'],x['end_time'],x['utter']), axis=1)

In [13]:
df.to_csv("videoplayback1_emo.csv")